<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(12_5_NLP%EB%A5%BC_%EC%9C%84%ED%95%9C_%EC%8B%A0%EA%B2%BD%EB%A7%9D(CNN)_Multi_Kernel_1D_CNN%EC%9C%BC%EB%A1%9C_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Kernel 1D CNN으로 네이버 영화 리뷰 분류하기

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 데이터 로드

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [ ]:
print("훈련용 데이터 개수: ", train_data.shape[0])
print("테스트용 데이터 개수: ", test_data.shape[0])

In [ ]:
train_data.head()

In [ ]:
test_data.head()

## 데이터 정제

In [ ]:
train_data.nunique()

In [ ]:
train_data = train_data.drop_duplicates(subset = ["document"])
train_data.shape

In [ ]:
train_data.label.value_counts().reset_index(name = "count")
train_data.label.value_counts().plot.bar()

In [ ]:
train_data.isnull().any()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data[train_data.document.isnull()]

In [ ]:
train_data = train_data.dropna(how = 'any')
train_data.shape

In [ ]:
train_data["document"] = train_data.document.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data.head()

In [ ]:
train_data["document"] = train_data["document"].str.replace("^ +", "")
train_data.head()

In [ ]:
train_data["document"] = train_data["document"].replace("",np.nan)
train_data.isnull().sum()

In [ ]:
train_data[train_data["document"].isnull()].head()

In [ ]:
train_data = train_data.dropna(how = 'any')
train_data.shape

In [ ]:
print(test_data.shape)
test_data = test_data.drop_duplicates(subset = ['document'])
test_data["document"] = test_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
test_data["document"] = test_data["document"].str.replace("^ +", "")
test_data["document"] = test_data["document"].replace("", np.nan)
test_data = test_data.dropna(how = 'any')
print(test_data.shape)

## 토큰화

In [ ]:
okt = Okt()
train_data["tokenized"] = train_data["document"].apply(lambda x : okt.morphs(x,stem = True))

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
train_data["tokenized"] = train_data["tokenized"].map(lambda x : [word for word in x if not word in stopwords])

In [ ]:
train_data["tokenized"][:5].values

In [ ]:
test_data["tokenized"] = test_data["document"].apply(lambda x : okt.morphs(x,stem = True))
test_data["tokenized"] = test_data["tokenized"].map(lambda x : [word for word in x if not word in stopwords])

In [ ]:
test_data["tokenized"][:5].values

## 정수인코딩

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data["tokenized"])

In [ ]:
print(tokenizer.word_index)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index)
total_freq = 0

rare_cnt = 0
rare_freq = 0
for key,value in tokenizer.word_counts.items():
  total_freq += value
  if value < 3:
    rare_cnt += 1
    rare_freq += value

print("단어 집합 크기: ",total_cnt)
print(f"등반 빈도가 {threshold} 미만인 희귀 단어 수: {rare_cnt}")
print(f"전체 단어 중 희귀 단어 비율: {rare_cnt / total_cnt * 100:.3f}")
print(f"전체 등장 빈도 중 희귀 단어 비율: {rare_freq / total_freq * 100:.3f}")

In [ ]:
vocab_size = total_cnt - rare_cnt + 1

In [ ]:
vocab_size

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_data["tokenized"])

X_train = tokenizer.texts_to_sequences(train_data["tokenized"])
X_test = tokenizer.texts_to_sequences(test_data["tokenized"])

In [ ]:
y_train = np.array(train_data["label"])
y_test = np.array(test_data["label"])

## 빈 샘플 제거

In [ ]:
drop_train = [idx for idx, sentence in enumerate(X_train) if len(sentence) < 1]

#빈 샘플 제거
X_train = np.delete(X_train, drop_train, axis = 0)
y_train = np.delete(y_train, drop_train, axis = 0)

print(len(X_train), len(y_train))

## 패딩

In [ ]:
max_len = max([len(s) for s in X_train])
len_list = [len(s) for s in X_train]
for length in range(1,max_len+1):
  cnt = 0
  for s in X_train:
    if len(s) <= length:
      cnt +=1
  print(f"샘플 중 길이가 {length} 이하인 샘플의 비율: {cnt / len(len_list)*100:.3f}")

In [ ]:
max_len = 30
X_train = pad_sequences(sequences=X_train, maxlen = max_len)
X_test = pad_sequences(sequences=X_test, maxlen = max_len)

## Multi-Kernel 1D CNN으로 네이버 영화 리뷰 분류하기

In [ ]:
print(X_train.shape)
X_train

In [ ]:
print(y_train.shape)
y_train

In [ ]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, Conv1D, Input, Flatten, Concatenate


In [ ]:
# 하이퍼 파라미터 정의
embedding_dim = 128
dropout_prob = (0.5, 0.8)
num_filters = 128

In [ ]:
# 입력층과 임베딩층 정의
# 임베딩층 이후 50% 드랍아웃

model_input = Input(shape = (max_len,))
print(model_input)
z = Embedding(input_dim = vocab_size, output_dim = embedding_dim,input_length = max_len, name = "embedding")(model_input)
print(z)
z = Dropout(dropout_prob[0])(z)
print(z)

In [ ]:
conv_blocks = []

for sz in [3,4,5]:
  conv = Conv1D(filters = num_filters,
                kernel_size = sz,
                padding = "valid",
                activation = "relu",
                strides = 1)(z)
  print("Con1D",sz,conv)
  conv = GlobalMaxPooling1D()(conv)
  print(conv)
  conv = Flatten()(conv)
  print(conv)
  conv_blocks.append(conv)
  print("*"*100)

In [ ]:
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
print(z)
z = Dropout(dropout_prob[1])(z)
print(z)
z = Dense(units = 128, activation = "relu")(z)
print(z)
model_output = Dense(units = 1, activation = 'sigmoid')(z)
print(model_output)

model = Model(model_input, model_output) #훈련 및 예측을 위해 여러 개 layer를 하나의 객체에 저장
print(model)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('CNN__model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
model.fit(x = X_train, y = y_train, batch_size = 64, epochs = 10, validation_data=(X_test,y_test),verbose = 2,callbacks = [es,mc])

In [ ]:
loaded_model = load_model(filepath = "CNN_model.h5")

print(f"\n테스트 정확도: {loaded_model.evaluate(x = X_test,y = y_test)[1]:.3f}%")

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('CNN_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
model.fit(x = X_train, y = y_train, batch_size = 64, epochs = 10, validation_data=(X_test,y_test),verbose = 2,callbacks = [es,mc])

In [ ]:
loaded_model = load_model(filepath = "CNN_model.h5")

print(f"\n테스트 정확도: {loaded_model.evaluate(x = X_test,y = y_test)[1]:.3f}%")

## 리뷰 예측

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence,stem=True) #토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] #불용어 처리
  encoded = tokenizer.texts_to_sequences([new_sentence]) #정수화
  pad_new = pad_sequences(encoded,maxlen = max_len)
  score = float(loaded_model.predict(pad_new))
  if score > 0.5 :
    print(f"{score*100:.3f}% 확률로 긍정 리뷰입니다.")
  else:
    print(f"{(1-score)*100:.3f}% 확률로 부정 리뷰입니다.")
  

In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

In [ ]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

In [ ]:
sentiment_predict('감독 뭐하는 놈이냐?')

In [ ]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

In [ ]:
# import tensorflow as tf
# x = np.arange(20).reshape(2, 2, 5)
# print(x)
# print("*"*100)
# y = np.arange(20, 40).reshape(2, 2, 5)
# print(y)
# print("*"*100)
# print(tf.keras.layers.Concatenate(axis=-1)([x, y]))
# print("*"*100)
# print(tf.keras.layers.Concatenate(axis=1)([x, y]))
# print("*"*100)
# print(tf.keras.layers.Concatenate(axis=0)([x, y]))
# print("*"*100)

In [ ]:
print(tf.keras.layers.Concatenate(axis=-1)([x, y]))
Flatten()(tf.keras.layers.Concatenate(axis=-1)([x, y]))